<a href="https://colab.research.google.com/github/LukasH-sys/Combinatorial_Optimization_Exercises/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

A = np.loadtxt('small_instance_A.txt')
c = np.loadtxt('small_instance_c.txt')
labda = np.array([1,1,1,1,1,1])



import numpy as np

def Lagrangian(c: np.ndarray, A: np.ndarray, labda: np.ndarray) -> tuple[float, np.ndarray]:

    obj_lagrange = np.sum(labda)
    x_lagrange = np.zeros((A.shape[1], 1))

    for j in range(A.shape[1]):

        constraint_correction = np.sum(labda[:,0] * A[:, j])

        if c[j] - constraint_correction < 0:
            x_lagrange[j] = 1
        else:
            x_lagrange[j] = 0

        obj_lagrange = obj_lagrange + min(c[j] - constraint_correction, 0)

    return obj_lagrange , x_lagrange


def InfeasToFeas(c: np.ndarray, A: np.ndarray, x_infeas: np.ndarray) -> tuple[float, np.ndarray]:
    x_feas = np.copy(x_infeas)

    S = {j for j in range(A.shape[1]) if x_feas[j] == 1}

    C = {i for i in range(A.shape[0]) if np.dot(A[i, :], x_feas) >= 1}

    # start algorithm
    while len(C) < A.shape[0]:

        # initialise utility ratio
        best_ratio = -np.inf
        best_j = -1

        # Select route
        for j in range(A.shape[1]):

            if j not in S:
                uncovered_constraint = {i for i in range(A.shape[0]) if i not in C and A[i, j] > 0}

                if len(uncovered_constraint) > 0:
                    ratio = len(uncovered_constraint) / c[j]
                    if ratio > best_ratio:
                        best_ratio = ratio
                        best_j = j

        # Update solution
        if best_j != -1:
            x_feas[best_j] = 1

        S.add(best_j)
        C.update({i for i in range(A.shape[0]) if np.dot(A[i, :], x_feas) >= 1})

    # Calculate objective function
    obj_feas = np.sum(c[list(S)])

    return obj_feas, x_feas

def UpdateLabda(A: np.ndarray, labda: float, LB: float, UB: float, x_lagrange: np.ndarray, rho: float):
    d = np.ones(A.shape[0])
    d_column_vector = d[:, np.newaxis]
    dotprod = np.dot(A, x_lagrange)

    dif_const = d_column_vector - dotprod

    if np.linalg.norm(dif_const) ** 2 == 0:
        return labda

    mu_k = rho * (UB - LB) / np.linalg.norm(dif_const) ** 2

    labda_next = labda + mu_k * (dif_const)

    labda_next = np.maximum(labda_next, 0)

    return labda_next

def SubgradientOpt(c: np.ndarray, A: np.ndarray, labda_init: np.ndarray, rho_init: float, k: int) -> tuple[float, float, np.ndarray, list, list]:
    labda = np.copy(labda_init)
    UB_best = np.inf
    LB_best = -np.inf
    x_best = None
    UB_list = []
    LB_list = []

    for iteration in range(1, k + 1):

        obj_l, x_lagrange = Lagrangian(c, A, labda)

        LB = obj_l
        LB_list.append(LB)
        if LB > LB_best:
            LB_best = LB

        Obj_i, x_feasible = InfeasToFeas(c, A, x_lagrange)

        UB = Obj_i
        UB_list.append(UB)
        if UB < UB_best:
            UB_best = UB
            x_best = x_feasible

        rho = rho_init / iteration

        labda_next = UpdateLabda(A, labda, LB, UB, x_lagrange, rho)

        labda = labda_next

    return LB_best, UB_best, x_best, LB_list, UB_list

################################################################################


LBbest, UBbest, xbest, listl, listu = SubgradientOpt(c, A, np.ones((A.shape[0],1)), 0.5, 50)
print(LBbest)
print(UBbest)
print(xbest)
print(listl)
print(listu)

Large_A = np.loadtxt('large_instance_A.txt')
Large_c = np.loadtxt('large_instance_c.txt')

LBbestlarge, UBbestlarge, xbestlarge, listllarge, listularge = SubgradientOpt(Large_c, Large_A, np.ones((Large_A.shape[0],1)), 200, 1000)
print(LBbestlarge)
print(UBbestlarge)
print(listllarge)
print(listularge)



6.488727054479679
7.0
[[0.]
 [0.]
 [1.]
 [0.]
 [1.]]
[4.0, 4.2, 5.15, 5.625000000000001, 5.886979166666667, 5.998281250000001, 6.0817578125, 6.147346540178572, 6.200637381417411, 6.2450464157831105, 6.282794094993955, 6.313020833333335, 6.315394363403321, 6.341644965277777, 6.324484371003651, 6.35103376958736, 6.355849224928194, 6.371313964287756, 6.374794835959719, 6.382680209156331, 6.384508770972614, 6.395516297741611, 6.399163324044695, 6.409135249947165, 6.406701154975605, 6.414778399639712, 6.418567131876093, 6.426032661185102, 6.42933440721172, 6.431309629702467, 6.434815995905519, 6.440639551200481, 6.443931866939302, 6.448653330442792, 6.448554821414122, 6.453181878481495, 6.456432609679635, 6.460776574613697, 6.463778114954236, 6.464059060165022, 6.467452877548114, 6.471071496642559, 6.4739473546511865, 6.476943721482013, 6.4771796695910995, 6.480252840824743, 6.482988784373421, 6.485902266467953, 6.488488903688598, 6.488727054479679]
[10.0, 8.0, 8.0, 8.0, 7.0, 7.0, 7.0, 7.0,